In [2]:
import tensorflow as tf

# Losses

## RPN

### Shared inputs


In [14]:
from od.core.anchor_generator import generate_anchors
from od.core.argmax_matcher import ArgMaxMatcher
from od.core.box_coder import encode_boxes_faster_rcnn
from od.core.box_ops import compute_iou
from od.core.losses import SmoothL1Localization
from od.core.sampling_ops import batch_sample_balanced_positive_negative
from od.core.standard_fields import BoxField, LossField
from od.core.target_assigner import TargetAssigner, batch_assign_targets

RPN_BATCH_PER_IM = 256
matcher = ArgMaxMatcher(0.7, 0.3, force_match_for_each_row=True)
target_assigner = TargetAssigner(compute_iou, matcher, encode_boxes_faster_rcnn)

localization_pred = tf.constant(
        [[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]],
        tf.float32)

classification_pred = tf.constant(
        [[[-100, 100], [100, -100], [100, -100]], [[-100, 100], [-100, 100], [-100, 100]]],
        tf.float32)

anchors = tf.constant([[0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]], tf.float32)

ground_truths = {
        BoxField.BOXES:
            tf.constant([[[0, 0, 1, 1], [0, 0, 2, 2]], [[0, 0, 3, 3], [0, 0, 0, 0]]], tf.float32),
        BoxField.NUM_BOXES:
            tf.constant([[2], [1]], tf.int32),
    }

# Will be auto batch by the target assigner
anchors = {BoxField.BOXES: anchors}

# We only want the Localization field here the target assigner will understand that
# it is the RPN mode.
gt_boxes = tf.unstack(ground_truths[BoxField.BOXES])
num_boxes = tf.unstack(ground_truths[BoxField.NUM_BOXES])
ground_truths = [{BoxField.BOXES: b[:nb[0]]} for b, nb in zip(gt_boxes, num_boxes)]

y_true, weights, _ = batch_assign_targets(target_assigner, anchors, ground_truths)

## Samling
# y_true[LossField.CLASSIFICATION] is a [batch_size, num_anchors, 1]
labels = tf.cast(y_true[LossField.CLASSIFICATION][:, :, 0], dtype=bool)
sample_idx = batch_sample_balanced_positive_negative(
    weights[LossField.CLASSIFICATION],
    RPN_BATCH_PER_IM,
    labels,
    positive_fraction=0.5)

weights[LossField.CLASSIFICATION] = tf.multiply(sample_idx,
                                                        weights[LossField.CLASSIFICATION])
weights[LossField.LOCALIZATION] = tf.multiply(sample_idx, weights[LossField.LOCALIZATION])



In [19]:
y_true[LossField.CLASSIFICATION][:, :, 0]

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

In [15]:
weights

{'classification': <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 1., 0.],
        [1., 0., 1.]], dtype=float32)>,
 'localization': <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 1., 0.],
        [0., 0., 1.]], dtype=float32)>}

In [29]:
label_logits = tf.nn.softmax(classification_pred)[:, :, 0]
label_logits

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 1., 1.],
       [0., 0., 0.]], dtype=float32)>

### Tensorpack

In [37]:
anchor_labels = y_true[LossField.CLASSIFICATION][:, :, 0]
y_pred_bce = tf.nn.softmax(classification_pred)[:, :, 0]
valid_mask = tf.stop_gradient(tf.not_equal(anchor_labels, -1))
pos_mask = tf.stop_gradient(tf.equal(anchor_labels, 1))
nr_valid = tf.stop_gradient(tf.math.count_nonzero(valid_mask, dtype=tf.int32), name='num_valid_anchor')
nr_pos = tf.identity(tf.math.count_nonzero(pos_mask, dtype=tf.int32), name='num_pos_anchor')

valid_anchor_labels = tf.boolean_mask(anchor_labels, valid_mask)
valid_label_logits = tf.boolean_mask(label_logits, valid_mask)


# Per-level loss summaries in FPN may appear lower due to the use of a small placeholder.
# But the total RPN loss will be fine.  TODO make the summary op smarter
placeholder = 0.

label_loss = tf.keras.losses.binary_crossentropy(valid_anchor_labels, valid_label_logits, from_logits=False)
#label_loss = tf.reduce_sum(label_loss) * (1. / RPN_BATCH_PER_IM)
#label_loss = tf.where(tf.equal(nr_valid, 0), placeholder, label_loss, name='label_loss')
label_loss


<tf.Tensor: shape=(), dtype=float32, numpy=7.6971893>

In [38]:
label_loss * 256

<tf.Tensor: shape=(), dtype=float32, numpy=1970.4805>

In [34]:
valid_anchor_labels

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([1., 1., 0., 0., 0., 1.], dtype=float32)>

In [35]:
valid_label_logits

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 0.], dtype=float32)>

In [ ]:
pos_anchor_boxes = tf.boolean_mask(anchor_boxes, pos_mask)
pos_box_logits = tf.boolean_mask(box_logits, pos_mask)
delta = 1.0 / 9
box_loss = tf.losses.huber_loss(
    pos_anchor_boxes, pos_box_logits, delta=delta,
    reduction=tf.losses.Reduction.SUM) / delta
box_loss = box_loss * (1. / RPN_BATCH_PER_IM)
box_loss = tf.where(tf.equal(nr_pos, 0), placeholder, box_loss, name='box_loss')


### OD

In [ ]:
localization_pred = tf.constant(
        [[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]],
        tf.float32)

classification_pred = tf.constant(
        [[[-100, 100], [100, -100], [100, -100]], [[-100, 100], [-100, 100], [-100, 100]]],
        tf.float32)

anchors = tf.constant([[0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]], tf.float32)

ground_truths = {
        BoxField.BOXES:
            tf.constant([[[0, 0, 1, 1], [0, 0, 2, 2]], [[0, 0, 3, 3], [0, 0, 0, 0]]], tf.float32),
        BoxField.NUM_BOXES:
            tf.constant([[2], [1]], tf.int32),
    }
rpn = RegionProposalNetwork(classification_loss_weight=1.0)
losses = rpn.compute_loss(localization_pred, classification_pred, anchors, ground_truths)

assert losses[LossField.CLASSIFICATION] == 100
assert losses[LossField.LOCALIZATION] == 0
assert len(losses) == 2


In [ ]:

# Create one_hot encoding [batch_size, num_anchors, 1] -> [batch_size, num_anchors, 2]
y_true[LossField.CLASSIFICATION] = tf.one_hot(tf.cast(
    y_true[LossField.CLASSIFICATION][:, :, 0], tf.int32),
                                              depth=2)

weights[LossField.CLASSIFICATION] = tf.multiply(sample_idx,
                                                weights[LossField.CLASSIFICATION])
weights[LossField.LOCALIZATION] = tf.multiply(sample_idx, weights[LossField.LOCALIZATION])

y_pred = {
    LossField.CLASSIFICATION: classification_pred,
    LossField.LOCALIZATION: localization_pred
}

return self.compute_losses(y_true, y_pred, weights)
